<a href="https://colab.research.google.com/github/LazyHooks/Optimus/blob/main/Optimus%20Predictor_v6.16%20-%20GCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

In [3]:
from pickle import load
import pandas as pd
from scipy.sparse import hstack
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

Load Data

In [49]:
#Import the Payroll Data
df_test = pd.read_excel('test1.xlsx')

#Import the sector reference table
ref_table = pd.read_excel('GCC_ref.xlsx')

df_test.shape

(104068, 6)

Load Models

In [23]:
#Load Models
model_JF=load(open('GCCJF.pkl', 'rb'))
model_Level=load(open('GCCLevel.pkl', 'rb'))

#Load Supporting Elements
vectorizers_JF=load(open('GCCJF_vect.pkl', 'rb'))
vectorizers_Level=load(open('GCCLevel_vect.pkl', 'rb'))

Static Variables

In [24]:
train_cols = ['Co. Designation', 'Co. Function']

In [25]:
target_col = 'Job Family Code'
target_col_2='Level'
jfcol='Job Family Code'

In [26]:
all_cols = ['Co. Designation', 'Co. Function', 'Job Family Code','Level']
univ_cols= ['Co. Designation', 'Co. Function', 'Job Family Code','Level','TFP']

In [27]:
df_test_2=df_test
df_test_2=df_test[univ_cols]
df_test = df_test[all_cols]

Data Preprocessing JF

In [28]:
X_test_unseen = []

for i, col in enumerate(train_cols):
    vectorizer = vectorizers_JF[i]
    X_test_col = vectorizer.transform(df_test.loc[:, col])
    X_test_unseen.append(X_test_col)

X_unseen = hstack(X_test_unseen)

JF Predictions

In [29]:
y_test_pred = model_JF.predict (X_unseen)

In [31]:
y_test_pred_copy=pd.DataFrame(y_test_pred)
y_test_pred_copy.columns = ['Job Family Code']

Data Preprocessing Level

In [33]:
X_test_unseen_2 = []
z=0
for i, col in enumerate(train_cols):
    vectorizer = vectorizers_Level[i]
    X_test_col_2 = vectorizer.transform(df_test.loc[:, col])
    X_test_unseen_2.append(X_test_col_2)
    z=i

vectorizer = vectorizers_Level[z+1]
X_test_col_2 = vectorizer.transform(y_test_pred_copy.loc[:, jfcol])
X_test_unseen_2.append(X_test_col_2)

X_unseen_2 = hstack(X_test_unseen_2)


Level Predictions

In [34]:
y_test_pred_2 = model_Level.predict (X_unseen_2)

In [35]:
pd.DataFrame(y_test_pred).to_csv('JF_x.csv')
pd.DataFrame(y_test_pred_2).to_csv('Lvl_x.csv')
frames=[pd.DataFrame(y_test_pred),pd.DataFrame(y_test_pred_2)]
maps=pd.concat(frames, axis=1)

Level Correction Preprocessing

In [36]:
temp=df_test_2.loc[:,'TFP']
init_maps=pd.concat([maps , temp], axis=1)
init_maps

,0,0,TFP
0,FLXX,E1,23940755.0
1,BSMT,M6,29699480.0
2,PRPO,M6,23540580.0
3,BFCB,M6,24483710.0
4,BSMT,M6,20382135.0
...,...,...,...
104063,BFCB,S3,357866.0
104064,SAAC,S3,323780.0
104065,BFCB,S3,352111.0
104066,BFCB,S3,310834.0


In [37]:
init_maps.columns = ['JF', 'level', 'TFP']

In [38]:
def find_in_ref(df, jf, level):
  try:
    return df.loc[(df.JF == jf) & (df.Level == level)]
  except:
    return None

def check_range_ref(ref_df, comp, index):
  ref_row = ref_df.loc[ref_df.index == index]

  if comp >= ref_row.Min.values[0] and comp <= ref_row.Max.values[0] :
    return True

  return False

def compare_range_ref(ref_df, comp, index):
  ref_row = ref_df.loc[ref_df.index == index]

  if comp < ref_row.Min.values[0]:
    return -1
  elif comp > ref_row.Max.values[0]:
    return 1
  elif comp >= ref_row.Min.values[0] and comp <= ref_row.Max.values[0] :
    return 0

  return False


In [39]:
reassigned_maps = init_maps.copy(deep=True)

Level Correction

In [40]:
from tqdm.auto import tqdm

In [43]:
ref_table.loc[ref_table.index == 0].JF.values[0]

'AFAD'

In [50]:
for index, query in tqdm(init_maps.iterrows()):
  ref_row = find_in_ref(ref_table, query.JF, query.level)
  d_index=index

  try:
    ref_index = ref_row.index[0]
  except:
    continue

  # check if comp falls in that range
  comp = query['TFP']/1000
  final_index = -1
  cur_jf=query.JF
  if compare_range_ref(ref_table, comp, ref_index)==0:
    #print("Exact match found at ", ref_index)
    final_index = ref_index
  elif compare_range_ref(ref_table, comp, ref_index )==-1:
    ref_index_l=ref_index
    while ref_table.loc[ref_table.index == ref_index_l].JF.values[0]==cur_jf:
      if ref_table.loc[ref_table.index == ref_index_l].Level.values[0]=='S2':
        final_index = ref_index_l
        break
      elif compare_range_ref(ref_table, comp, ref_index_l)==0:
        final_index = ref_index_l
        break
      ref_index_l=ref_index_l-1
  elif compare_range_ref(ref_table, comp, ref_index )==1:
    ref_index_u=ref_index
    while ref_table.loc[ref_table.index == ref_index_u].JF.values[0]==cur_jf:
      if compare_range_ref(ref_table, comp, ref_index_u)==0:
        final_index = ref_index_u
        break
      elif ref_table.loc[ref_table.index == ref_index_u].Level.values[0]=='E2':
        final_index = ref_index_u
        break
      else:
        if ref_table.loc[ref_table.index == ref_index_u].Level.values[0]=='I7':
          ref_index_u = ref_index_u+7
        else:
          ref_index_u = ref_index_u+1

  if final_index != -1 :
    final_level = ref_table.loc[ref_table.index == final_index].Level.values[0]
    reassigned_maps.at[d_index, 'level'] = final_level


0it [00:00, ?it/s]

Final Outputs

In [47]:
pd.DataFrame(init_maps).to_csv('init_maps.csv')
pd.DataFrame(reassigned_maps).to_csv('reassigned maps.csv')